<a href="https://colab.research.google.com/github/ashishar/qbook/blob/master/Solution_PI_QML_Tutorial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantum Machine Learning - Tutorial 1**
# **Quantum Circuits and Optimization**

**Please do not modify this notebook. To work on the tutorial, create your own copy by going to File > Save a Copy in Drive**

Before doing anything, please **run the following cell to install PennyLane.**

In [ ]:
%%capture
!pip install pennylane

In this tutorial we will learn how to build quantum circuits in PennyLane and use them for various optimization tasks. You do not need to solve the entire tutorial in 1.5 hours. Choose the topics that you find will benefit your learning and interests the most.

First, let's import PennyLane and some other libraries. **Run the cell below.**

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import random

## **1. Circuit building**

### **A. Wax on, wax off**

Let's warm up by building some quantum circuits using PennyLane. Build the following circuits using PennyLane and return the indicated output. Don't forget to define a device and decorate your circuit to make it a QNode!

(i) Output: Computational basis measurement probabilities on the first wire.

<p align="center">
  <img width="50%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/circuit_building_1.png" />
</p>

In [ ]:
dev = qml.device('default.qubit', wires = [0,1])

@qml.qnode(dev)
def circuit():
  qml.RX(np.pi/3, wires = 0)
  qml.Hadamard(wires = 0)
  qml.CNOT(wires = [0,1])
  qml.Hadamard(wires = 0)

  return qml.probs(wires =[0])

circuit()

tensor([0.5, 0.5], requires_grad=True)

(ii) Output: Expectation value of the Hamiltonian $ H = 2X \otimes X + 5 Y \otimes Y $

<p align="center">
  <img width="50%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/circuit_building_2.png" />
</p>


In [ ]:
dev = qml.device('default.qubit', wires = [0,1])

coeffs = [2, 5]
obs = [qml.PauliX(0)@qml.PauliX(1), qml.PauliY(0)@qml.PauliY(1)]
H = qml.Hamiltonian(coeffs, obs)

@qml.qnode(dev)
def circuit():

  qml.Hadamard(wires = 0)
  qml.Hadamard(wires = 1)
  qml.CY(wires = [0,1])
  qml.RY(np.pi/4, wires = 0)

  return qml.expval(H)

circuit()

tensor(1.41421356, requires_grad=True)

(iii) Output: Full two-qubit quantum state at the end of the circuit

<p align="center">
  <img width="50%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/circuit_building_3.png" />
</p>

In [ ]:
dev = qml.device('default.qubit', wires = [0,1])

@qml.qnode(dev)
def circuit():
  qml.RY(np.pi/4, wires = 0)
  qml.RX(np.pi/6, wires = 1)
  qml.PauliX(wires = 1)
  qml.CRZ(np.pi/3, wires = [1,0])
  qml.PauliX(wires = 1)
  qml.PauliX(wires = 0)

  return qml.state()

circuit()

tensor([0.32012093+0.18482191j, 0.        -0.09904576j,
        0.77284029-0.44619955j, 0.        -0.23911762j], requires_grad=True)

(iv) Output: Reduced density matrix on the first two wires.

<p align="center">
  <img width="50%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/circuit_building_4.png" />
</p>

In [ ]:
dev = qml.device('default.qubit', wires = [0,1,2])

@qml.qnode(dev)
def circuit():
  qml.RY(np.pi/4, wires = 0)
  qml.RX(np.pi/6, wires = 1)
  qml.Hadamard(wires = 2)
  qml.Toffoli(wires=[2,1,0])
  qml.CY(wires = [2,0])

  return qml.density_matrix(wires = [0,1])

circuit()

tensor([[0.46650635+0.j        , 0.        +0.15088835j,
         0.        +0.j        , 0.        +0.02588835j],
        [0.        -0.15088835j, 0.05717724+0.j        ,
         0.        +0.0625j    , 0.        +0.j        ],
        [0.        +0.j        , 0.        -0.0625j    ,
         0.46650635+0.j        , 0.        +0.0625j    ],
        [0.        -0.02588835j, 0.        +0.j        ,
         0.        -0.0625j    , 0.00981006+0.j        ]], requires_grad=True)

### **B. Deutsch-Jozsa algorithm**

**Do this exercise if you're interested in applying your circuit building skills to construct some standard quantum algorithms.**

Now let's apply our circuit-building skills to implement oracle-based quantum algorithms. Let's start with Deutsch-Jozsa.

Recall that the Deustch-Jozsa Algorithm tells us whether a function $f:\{0,1\}^n \rightarrow \{0,1\}$ is constant or balanced. Here, balanced means that $|f^{-1}(0)| = |f^{-1}(1)|.$ This function is encoded in a "black box oracle", which is a unitary $U_f$ such that $U_f(\vert \mathbf{x} \rangle) = \vert \mathbf{x}\rangle$ if $f(\mathbf{x}) = 0,$ and $U_f(\vert \mathbf{x} \rangle) = -\vert \mathbf{x}\rangle$ if $f(\mathbf{x}) = 1.$

You are asked to implement the Deutsch-Jozsa algorithm to determine whether the functions that a randomly generated oracle encodes are constant or balanced. Do some Google research if you don't remember/ever learned how Deutsch-Jozsa works!

The following `generate_oracle` function generates the matrix of a random oracle using a seed. All oracles are assumed to be represented by $16 \times 16$ matrices (so we're working with $n=4$ qubits). **Run the cell below.**

In [ ]:
def generate_oracle(seed):

  random.seed(seed)
  j = random.randint(0,1)
  k = random.randint(0,1)
  length = 8
  array = [j] * length + [k] * length
  random.shuffle(array)

  oracle = np.identity(16)
  for i in range(len(array)):
    if array[i]==1:
      oracle[i][i] = -1
  return oracle

Now determine whether the a given seed generates an oracle associated with constant or balanced function using the Deutsch-Jozsa Algorithm shown below. **Complete the cell below with your solution.**

<p align="center">
  <img width="50%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/dj.png" />
</p>

*Hint:* Combine PennyLane's `qml.QubitUnitary` with the `generate_oracle` function defined above.

In [ ]:
dev = qml.device('default.qubit', wires = range(4))

@qml.qnode(dev)
def deutsch_jozsa(seed):

  qml.broadcast(qml.Hadamard, wires = range(4), pattern = "single")
  qml.QubitUnitary(generate_oracle(seed), wires = range(4))
  qml.broadcast(qml.Hadamard, wires = range(4), pattern = "single")

  return qml.probs(wires = range(4))

### **C. Grover's algorithm**

**Do this exercise if you're interested in applying your circuit building skills to construct some standard quantum algorithms.**

Now let's code the famous Grover Search Algorithm! For a function $f:\{0,1,\ldots, N-1\}\rightarrow \{0,1\}$ Grover's algorithm determines the set $f^{-1}(1)$; that is, it finds the solutions to the equation $f(s) = 1$. In this problem, we will assume that $|f^{-1}(1)| = 1$, so that only one such solution $s$ exists.

The code below uses a `seed` to generate the binary representation of a random solution $s.$ For $N=16$, your task is to use Grover's Algorithm to find the solution $s$. **Run the cell below.**

In [ ]:
def generate_s(seed):

  random.seed(seed)
  length = 16
  k = random.randint(0,length-1)
  array = np.binary_repr(k, width = 4)

  return np.array([float(i) for i in array])

(i) First, let's build an oracle $U_f$ associated with the function $f$. The action of this oracle is $U_f\vert \mathbf{x}\rangle |y\rangle = |\mathbf{x}\rangle |y\oplus f(\mathbf{x})\rangle.$ Here, $y$ is an auxiliary qubit. As a gate, this oracle is the 5-qubit unitary shown below,

<p align="center">
  <img width="50%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/oracle.png" />
</p>

where the control values are given by the binary representation of the unique solution $s$. In the figure, it's $1001_2 = 9.$

**Complete the cell below** with a subcircuit that applies this oracle as a function of the `seed`. The function `generate_f` will be super useful for you.

*Hint:* Use the `qml.MultiControlledX` gate.

In [ ]:
def grover_oracle(seed):

  qml.MultiControlledX(wires = range(5), control_values=generate_s(seed))

(ii) Next, let's build the diffusion operator, which is the following 5-qubit unitary.

<p align="center">
  <img width="50%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/diffusion.png" />
</p>

In [ ]:
def diffusion():

  qml.broadcast(qml.Hadamard, wires = range(4), pattern = 'single')
  qml.MultiControlledX(wires = range(5), control_values = [0,0,0,0])
  qml.broadcast(qml.Hadamard, wires = range(4), pattern = 'single')

(iii) Finally, let's build the circuit for Grover search given below. What is the number of oracle-diffusion iterations needed to find the solution with maximal probability?

<p align="center">
  <img width="50%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/grover.png" />
</p>

In [ ]:
dev = qml.device('default.qubit', wires = range(5))

@qml.qnode(dev)
def grover_circuit(seed, num_iterations):

  qml.broadcast(qml.Hadamard, wires = range(4), pattern = 'single')
  qml.PauliX(wires = 4)
  qml.Hadamard(wires = 4)

  for _ in range(num_iterations):
    grover_oracle(seed)
    diffusion()

  return qml.probs(wires = range(4))

## **2. Circuit optimization**

### **A. Practice makes perfect**

Build each of the following parametric circuits in PennyLane as QNodes, write the cost functions specified, and find their minima. For reference, the code you need to optimize cost functions using gradient descent is given below.

In [ ]:
def optimize(cost_function, init_params, steps):

  opt = qml.GradientDescentOptimizer(stepsize = 0.4) # Change this as you see fit

  params = init_params

  for i in range(steps):

    params = opt.step(cost_function, params)

  return params, cost_function(params)

(i) Cost function: Expectation value of the Pauli-Z operator on the first wire

<p align="center">
  <img width="50%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/optimization_1.png" />
</p>

In [ ]:
dev = qml.device('default.qubit', wires = [0,1,2])

@qml.qnode(dev)
def circuit_1(params):

  qml.RX(params[0], wires = 0)
  qml.CNOT(wires=[0,1])
  qml.CNOT(wires = [0,2])
  qml.RY(params[1], wires = 0)
  qml.RY(params[2], wires = 1)
  qml.RY(params[3], wires = 2)

  return qml.expval(qml.PauliZ(0))

In [ ]:
def cost_function(params):
  return circuit_1(params)

Cost function: Minus the fidelity between the circuit's output state and the Bell state $\left\lvert \Phi_{+} \right\rangle = \frac{1}{\sqrt{2}} \left\lvert 00\right\rangle + \frac{1}{\sqrt{2}}\left\lvert 11 \right\rangle.$

<p align="center">
  <img width="30%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/optimization_2.png" />
</p>

In [ ]:
dev = qml.device('default.qubit', wires = [0,1])

@qml.qnode(dev)
def circuit_2(params):

  qml.RY(params[0], wires = 0)
  qml.CNOT(wires=[0,1])

  return qml.state()

In [ ]:
def cost_function(params):
  return -np.abs(np.dot(circuit_2(params), np.array([1/np.sqrt(2),0,0,1/np.sqrt(2)])))

In [ ]:
optimize(cost_function, np.array([0.], requires_grad = True),100)

(tensor([1.57075193], requires_grad=True), -0.9999999997535602)

(iii) Cost function: Minus the purity of the reduced density operator representing the output in the first two wires.

<p align="center">
  <img width="50%" src="https://raw.githubusercontent.com/gmlejarza/Tutorials_Perimeter/main/images/optimization_3.png" />
</p>

In [ ]:
dev =qml.device('default.qubit', wires = [0, 1 ,2])

@qml.qnode(dev)
def circuit_3(params):

  qml.Hadamard(wires = 0)
  qml.CRX(params[0], wires = [0,1])
  qml.CRY(params[1], wires = [1,2])
  qml.CNOT(wires = [0,2] )

  return qml.purity(wires = [0,1])


In [ ]:
def cost_function(params):
  return -1*circuit_3(params)

In [ ]:
optimize(cost_function, np.array([1.,1.], requires_grad = True), 100)

(tensor([3.1415833 , 3.13612877], requires_grad=True), -0.9999962682314241)

### **B. Application: Distinguishing Quantum States**

Suppose that you have prepared a quantum state, but you're not entirely sure which state it is. You know that you may have prepared

$$ \vert \phi_1 \rangle = \cos(\theta_1)\vert 0 \rangle + \sin(\theta_1)\vert 1 \rangle \quad \mathrm{with \ probability\ } p_1,$$
$$ \vert \phi_2 \rangle = \cos(\theta_2)\vert 0 \rangle + \sin(\theta_2)\vert 1 \rangle \quad \mathrm{with \ probability\ } p_2,$$

where $\theta_1$ ande $\theta_2$ are given. You are asked to devise an optimal strategy that will correctly identify the state with the highest probability possible. In this context, this means that you must maximize the average case success probability

$$ p_{success} = p_1\times q_1 + p_2\times q_2,$$

where $q_i = Prob(\text{state identified as } \vert \phi_i\rangle \vert \text{state was actually} \vert \phi_i\rangle).$ This maximum depends on the values of $\theta_1$ and $\theta_2.$

(Of course, you may compute this directly using the Holevo-Helstrom theorem, but try to use an optimization routine instead!)

In [ ]:
dev = qml.device('default.qubit', wires = 1)

@qml.qnode(dev)
def max_prob(theta, angle):

    qml.QubitStateVector([np.cos(theta), np.sin(theta)], wires = 0)

    qml.RY(angle, wires = 0)

    return qml.probs(wires = 0)

theta_1 = 0.0
theta_2 = 0.7853981633974483
p_1 = 0.25
p_2 = 0.75

def cost(angle):

    return 1-p_1*max_prob(theta_1, angle)[0]-p_2*max_prob(theta_2,angle)[1]

In [ ]:
1-optimize(cost, np.array(0.0, requires_grad = True), 500)[1]

tensor(0.89528471, requires_grad=True)

### **C. Application: The CHSH game**

The CHSH game provides an example of a quantum strategy that increases the probability of winning over any possible classical protocol. The game is collaborative between two parties, Alice and Bob, who are given two random bits $x$ and $y$, where $x$ is known by Alice and $y$ is known by Bob. Alice and Bob will additionally select two values, $a$ and $b$, which can be 0 or 1, and will win if

$$x\cdot y = a\oplus b, $$

where $\oplus$ represents addition modulo 2. Since the probability of $x\cdot y=0$  is 75%, the best classical strategy is that both of them previously agree on choosing either $a,b = 0,0$ or $1,1$, ignoring the bits they received.  

Can Alice and Bob do better if we provide them with an entangled pair of qubits? Let's consider the following quantum strategy. Alice and Bob are provided with the Bell pair

$$\left\lvert \Phi_{+} \right\rangle = \frac{1}{\sqrt{2}} \left\lvert 00\right\rangle + \frac{1}{\sqrt{2}}\left\lvert 11 \right\rangle.$$

With this state given to them, Alice and Bob devise a way to win the game by making conditional measurements in different bases.

Alice and Bob *separately* choose a measurement basis $\left\lbrace \left\lvert \nu_0(\theta) \right\rangle, \left\lvert \nu_1(\theta) \right\rangle\right\rbrace$, where

$$ \left\lvert \nu_0(\theta) \right\rangle = \cos(\theta)\left\lvert 0 \right\rangle +  \sin(\theta)\left\lvert 1 \right\rangle$$
and
$$ \left\lvert \nu_1(\theta) \right\rangle = -\sin(\theta)\left\lvert 0 \right\rangle + \cos(\theta)\left\lvert 1 \right\rangle. $$

For her basis, Alice chooses $\theta = \theta_{A0}$ if she receives $x=0$, and $\theta = \theta_{A1}$ if she receives $x=1$. Similarly, Bob will choose $\theta = \theta_{B0}$ if $y=0$ and $\theta = \theta_{B1}$ if $y=1$. Having chosen their angles that define their respective measurement bases, they measure $| \psi \rangle$! If Alice's measurement result corresponds to $\nu_0$ ($\nu_1$), she chooses $a = 0$ ($1$). Bob's choice is made in a similar fashion.

Your task is to build the quantum circuit that implements the measurements in the bases above and provide a set of angles that Alice and Bob should choose to maximize their probability of winning. Is the probability of winning is larger than in the classical setup?

(i) First, let's build a circuit that prepares the Bell Pair. **Complete the circuit below.**

In [ ]:
def prepare_entangled():

  qml.Hadamard(wires = 0)
  qml.CNOT(wires=[0, 1])


(ii) Now, complete the QNode `chsh_circuit` where Alice and Bob independently choose a measurement basis according to the results of their bits. Here `bit1` is the bit that Alice receives and `bit2` is the one that Bob receives. The measurement basis angles are enconded in the argument `params`, an array of the form $[\theta_{A0}, \theta_{A1}, \theta_{B0}, \theta_{B1}]$.

In [ ]:
dev = qml.device('default.qubit', wires = [0,1])

@qml.qnode(dev)
def chsh_circuit(params, bit1, bit2):

  prepare_entangled()

  if bit1 == 0:
      qml.RY(2 * params[0], wires=0)
      if bit2 == 0:
          qml.RY(2 * params[2], wires=1)
      else:
          qml.RY(2 * params[3], wires=1)
  else:
      qml.RY(2 * params[1], wires=0)
      if bit2 == 0:
          qml.RY(2 * params[2], wires=1)
      else:
          qml.RY(2 * params[3], wires=1)

  return qml.probs(wires=[0, 1])

(iii) Our cost function should be the probability that Alice and Bob win given their choice of basis. Define the function `winning_prob` that calculates the probability of winning given a choice of basis. Once again, the angles determining the basis are encoded in an array `params` of the form $[\theta_{A0}, \theta_{A1}, \theta_{B0}, \theta_{B1}]$.

In [ ]:
def winning_prob(params):
    prob00 = (
        chsh_circuit(params, 0, 0)[0]
        + chsh_circuit(params, 0, 0)[3]
    )
    prob01 = (
        chsh_circuit(params, 0, 1)[0]
        + chsh_circuit(params, 0, 1)[3]
    )
    prob10 = (
        chsh_circuit(params, 1, 0)[0]
        + chsh_circuit(params, 1, 0)[3]
    )
    prob11 = (
        chsh_circuit(params, 1, 1)[1]
        + chsh_circuit(params, 1, 1)[2]
    )
    return 1 / 4 * (prob00 + prob01 + prob10 + prob11)

(iv) Using the `winning_prob` function, define a cost function that we need to minimize.

In [ ]:
def cost(params):
    return 1 - winning_prob(params)

(v) Finally, optimize the `cost` function.

In [ ]:
optimize(cost, np.array([0.0, 0.3, 0.0, -0.5], requires_grad = True), 100 )

(tensor([-0.24634954,  0.53904862,  0.14634954, -0.63904862], requires_grad=True),
 tensor(0.14644661, requires_grad=True))